# Components

In [28]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

sys = """
Read the question and the four options: A, B, C and D.
Return a single letter "A", "B", "C" or "D" for the correct answer.
Do not return any other text.
"""

# gpt-4.1-nano
# gpt-4o-mini

def ask_gpt(prompt: str, model="gpt-4o-mini") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system", 
                "content": sys
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_tokens=1,
        n=1
    )
    return response.choices[0].message.content

In [29]:
import pandas as pd
from tqdm import tqdm

df = pd.read_json("MedQA.jsonl", lines=True)

correct = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    q = row['question']
    a = row['options']
    s = row['answer_idx']
    # QA = f"Q: {q}\nA: {a['A']}\nB: {a['B']}\nC: {a['C']}\nD: {a['D']}\nAnswer: {s}"
    # print(QA)
    prompt = f"Question: {q}\nA: {a['A']}\nB: {a['B']}\nC: {a['C']}\nD: {a['D']}\nAnswer:"
    while True:
        res = ask_gpt(prompt)
        if res in ['A', 'B', 'C', 'D']:
            break
    if res == s:
        correct += 1

accuracy = correct / len(df) * 100
print(f"Total: {len(df)}")
print(f"Correct: {correct}")
print(f"Accuracy: {accuracy:.2f}%")

100%|██████████| 1273/1273 [12:55<00:00,  1.64it/s]

Total: 1273
Correct: 941
Accuracy: 73.92%
